In [1]:
#import
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
#You will have to download the set of stop words the first time
#nltk.download('stopwords')
#nltk.download('punkt')
from collections import Counter

In [2]:
#define
top_N = 10000
stopwords = nltk.corpus.stopwords.words('english')
#we need to rename this directory to wherever the CSV files are located
yelp_attr = pd.read_csv('yelp_business_attributes.csv')
yelp_rev = pd.read_csv('yelp_review.csv',nrows = 10000)

In [3]:
#Combine Ambience and Reviews into a Dataframe
yelp_reviewd = pd.merge(yelp_rev, yelp_attr, on='business_id')
i = 0
for index, row in yelp_reviewd.iterrows():    
    yelp_reviewd.loc[index, "is_bougie"] = row["stars"] == 5
yelp_reviewd.head(10)
#list(yelp_reviewd.columns.values)



,review_id,user_id,business_id,stars,date,text,useful,funny,cool,AcceptsInsurance,...,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,is_bougie
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
1,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5,2016-02-22,*The shop was featured on Mind of a Chef with ...,4,3,4,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
2,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
3,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
4,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
5,i5kyjrSfzAC-RJLTYx6pTQ,pnASSACWj87icFTteMkbtA,ACFtxLv8pGrrxMm6EgjreA,5,2014-06-28,"Wow, just the most tasty burguer I've ever had...",0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
6,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
7,HRPm3vEZ_F-33TYVT7Pebw,_4iMDXbXZ1p1ONG297YEAQ,8QWPlVQ6D-OExqXoaD2Z1g,5,2014-09-24,Cycle Pub Las Vegas was a blast! Got a groupon...,1,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
8,ymAUG8DZfQcFTBSOiaNN4w,u0LXt3Uea_GidxRW1xcsfg,9_CGhHMz8698M9-PkVf0CQ,4,2012-05-11,Who would have guess that you would be able to...,0,0,2,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
9,8UIishPUD92hXtScSga_gw,u0LXt3Uea_GidxRW1xcsfg,gkCorLgPyQLsptTHalL61g,4,2015-10-27,Always drove past this coffee house and wonder...,1,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False


In [4]:
#Filter all restaurants by is_bougie true
is_bougie = yelp_reviewd[yelp_reviewd["is_bougie"] == True]
text_data_seriesb = is_bougie["text"]
text_data_seriesb.head(10)


0     Super simple place but amazing nonetheless. It...
1     *The shop was featured on Mind of a Chef with ...
2     Small unassuming place that changes their menu...
3     Lester's is located in a beautiful neighborhoo...
5     Wow, just the most tasty burguer I've ever had...
7     Cycle Pub Las Vegas was a blast! Got a groupon...
16    its delicious. its loud. its funny. seems like...
17    Love this place! If I ever have international ...
19    This place is awesome! Definitely authentic!!!...
28    Best burger place I've had in Toronto. Hands d...
Name: text, dtype: object

In [ ]:
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
wordsb = (text_data_seriesb
           .str.lower()
           .replace([r'\(','\)','\|','\,','\.','\?','\!','\_','\-','\:','\"','\*',"\'", RE_stopwords], ['','','','','','','',' ','','','','','', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rsltb = pd.DataFrame(Counter(wordsb).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rsltb)

# plot
#rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

            Frequency
Word                 
great            1834
place            1758
food             1462
good             1333
one              1135
time             1120
like             1088
get              1028
service           987
love              960
back              928
go                901
also              890
really            866
always            798
well              784
best              783
ive               726
would             707
delicious         692
nice              692
friendly          683
im                654
us                632
amazing           626
staff             618
dont              608
even              576
first             542
try               542
...               ...
courted             1
destined            1
flings              1
sensuality          1
begs                1
towners             1
suspicious          1
bendie              1
minded              1
gratuity            1
10pm12am            1
joked               1
stalks    

In [ ]:
#Repeat steps 5-8 with non-matching restaurants
#Filter all restaurants by is_bougie true
is_not_bougie = yelp_reviewd[yelp_reviewd["is_bougie"] == False]
text_data_seriesp = is_not_bougie["text"]
text_data_seriesp.head(10)

4     Love coming here. Yes the place always needs t...
6     Had their chocolate almond croissant and it wa...
8     Who would have guess that you would be able to...
9     Always drove past this coffee house and wonder...
10    Not bad!! Love that there is a gluten-free, ve...
11    I'm not on the Poutini's train. It's fine but ...
12    This is currently my parents new favourite res...
13    Server was a little rude.\n\nOrdered the calam...
14    I thought Tidy's Flowers had a great reputatio...
15    Wanted to check out this place due to all the ...
Name: text, dtype: object

In [ ]:
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
wordsp = (text_data_seriesp
           .str.lower()
           .replace([r'\(','\)','\|','\,','\.','\?','\!','\_','\-','\:','\"','\*',"\'", RE_stopwords], ['','','','','','','',' ','','','','','', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rsltp = pd.DataFrame(Counter(wordsp).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rsltp)

# plot
#rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

             Frequency
Word                  
good              4275
food              3808
place             3637
like              3261
one               2709
get               2505
great             2413
time              2365
would             2360
really            2282
service           2200
back              2031
go                1887
also              1883
dont              1661
im                1606
nice              1598
ordered           1508
little            1480
didnt             1455
well              1434
us                1375
got               1324
even              1316
order             1298
pretty            1293
came              1262
try               1225
menu              1220
ive               1205
...                ...
screamed             3
coquito              3
place;               3
billiards            3
cappuccinos          3
jeez                 3
davids               3
soirées              3
jouer                3
restuarant           3
130        

In [ ]:
odd_word = list({s for s in wordsb if s not in wordsp})
print(odd_word)


['everday', 'fhe', 'college\\/party', '10\\/10', 'rained', 'pamesan', 'neighborhoodbut', 'traci', 'greateven', 'incredibleask', 'hairdryers', 'replicated', 'induced', 'venenzias', 'tact', 'frenchies', 'standbys', 'vaz', 'mosturizer', 'snagandsavecom', 'starvin', 'calimari', 'alrightso', '$109', 'tratorria', 'vavaboom', 'meati', 'blindness', 'cramped;', 'lardons', 'photos\\/mudalytuatnzm2k9zoh27qselect=vanazcap5sz1twkuodhdpq', 'grandkid', 'yousef', 'hatter', 'barbera', 'muchas', 'leland', 'charla', 'fransisco', 'blueprint', 'salute', 'virus', 'pcl', 'chad', '$22\\/person', 'bonless', 'radical', 'kingpin', 'mags', 'wanda', 'kabocha', 'grohl', 'topmmmmmmmmm', 'zippy', 'vacuuming', '511', 'integrate', 'whichokaywas', 'mediating', 'duchess', 'ladopter', 'beauharnois', 'rositas', 'dingrédients', 'rics', 'historyfigures', 'asapto', 'coquette', 'turkeytasting', ';;', 'grape\\/bubble', '10a', 'tine', 'lunch;', 'jargons', 'ibach', 'betatted', 'oden', 'beerswe', 'coldpress', 'shamrockcactus', 'la

In [ ]:
odd_wordres = pd.DataFrame(Counter(odd_word).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(odd_wordres)


                  Frequency
Word                       
everday                   1
fhe                       1
college\/party            1
10\/10                    1
rained                    1
pamesan                   1
neighborhoodbut           1
traci                     1
greateven                 1
incredibleask             1
hairdryers                1
replicated                1
induced                   1
venenzias                 1
tact                      1
frenchies                 1
standbys                  1
vaz                       1
mosturizer                1
snagandsavecom            1
starvin                   1
calimari                  1
alrightso                 1
$109                      1
tratorria                 1
vavaboom                  1
meati                     1
blindness                 1
cramped;                  1
lardons                   1
...                     ...
reece                     1
yetis                     1
shiromi             

In [ ]:

# initilize by 0
# dictionary - business_id + counter
# increase counter for coorisponding business_id if you find a bougie word
# sort dictionary by values
# match review ID and resturant
business_id_to_counter = {}
for index, row in yelp_reviewd.iterrows():
    for word, word_row in odd_wordres.iterrows():
        if word in row["text"]:
            business_id = row["business_id"]
            # is the business in the dictionary already?
            if business_id in business_id_to_counter:
                business_id_to_counter[business_id] += 1
            else:
                # add the business
                business_id_to_counter[business_id] = 1

print(f"business_id_to_counter={business_id_to_counter}")
#sort the dictionary by values
sorted_biz = sorted(business_id_to_counter.items(), key=lambda kv: kv[1])
print(f"sorted_biz={sorted_biz}")